## <div align="center"> UJIAN TENGAH SEMESTER IF540-L MACHINE LEARNING </div>
## <div align="center"> Semester Gasal 2024/2025 </div>
## <div align="center"> [Judul Proyek] </div>

---
### Kelompok - 7

#### Anggota Kelompok : 
1. Leonardo Tyoes Huibu - 00000065503
2. Dylan William - 00000067644
3. Eduardus Farrel Tirtawinata - 00000069931
4. Emanuel Bernandhika Dwi Friskola - 00000077703


---

### Dataset yang digunakan untuk projek:

1. [Sentiment140 dataset with 1.6 million tweets – sumber : https://www.kaggle.com/datasets/kazanova/sentiment140

### Hasil kerja

In [ ]:
# Your codes are here (replace the following codes)
import time
time.sleep(10)

In [1]:
!jupyter nbconvert --to html "./UTS2024_IF540L_KelasE_Kelompok_7.ipynb" --output-dir="./"

[NbConvertApp] Converting notebook ./UTS2024_IF540L_KelasE_Kelompok_7.ipynb to html
[NbConvertApp] Writing 274232 bytes to UTS2024_IF540L_KelasE_Kelompok_7.html


### Next step:
* convert the generated html file to PDF using the online tool: https://www.sejda.com/html-to-pdf
* choose the following settings:
    * Page size: One long page
    * Page Orientation: auto
    * Use print stylesheet
* Submit your ipython notebook and PDF files

Markdown basics https://markdown-guide.readthedocs.io/en/latest/basics.html#